In [1]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings
from gcloud import storage

warnings.filterwarnings("ignore")
from google.cloud import texttospeech as tts

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\umair\Desktop\NeuroLeap\refined-veld-312721-eff81be2f58c.JSON'


In [3]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else: 
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output),f)

# tensorflow.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save("model.tflearn")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i,w in enumerate(words):
            if w == se:
                bag[i] = 1


    return numpy.array(bag)

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\umair\Desktop\Bott\Chatbot\model.tflearn


In [4]:
import sounddevice as sd
import soundfile as sf
import time
import os

storageclient = storage.Client()
bucket = storageclient.get_bucket('pibot')
blob = bucket.blob('inputsound.wav')

samplerate = 24000
duration = 2 # seconds
option = 'input'
def recording(word=None):
    filename = word + '.wav'
    print("start")
    mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
        channels=1, blocking=True)
    print("end")
    sd.wait()
    sf.write(filename, mydata, samplerate)
    blob.upload_from_filename(filename)

recording(option)

print("All recordings saved in {} ".format(os.getcwd()))

start
end
All recordings saved in C:\Users\umair\Desktop\Bott\Chatbot 


In [5]:
import os
import io
import google.cloud
from google.cloud import speech


def speechtotext():
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    # gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"
    gcs_uri = "gs://pibot/inputsound.wav"

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=24000,
        language_code="en-US",
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)

    output = " "
    for result in response.results:
        output = result.alternatives[0].transcript
        print("Transcript: {}".format(output))
        
    return output
    


In [8]:
def pibot():
    print("Start talking with the bot (type quit to stop)!")
    samplerate = 24000
    duration = 2 # seconds
    while True:
        initial = 'initial'
        recording(initial)
        initinput = speechtotext()
        if initinput.lower() == "start":
            print("You: ")
            option = 'inputsound'
            recording(option)
            inp = speechtotext()
            if inp.lower() == "quit":
                break

            results = model.predict([bag_of_words(inp.lower(), words)])[0]

            results_index = numpy.argmax(results)
            tag = labels[results_index]

            if results[results_index] > 0.99:
                for tg in data["intents"]:
                    if tg['tag'] == tag:
                        responses = tg['responses']

                print(random.choice(responses))
            else:
                print("I didn't get that, try again.")
        

pibot()


Start talking with the bot (type quit to stop)!
start
end
start
end
Transcript: start
You: 
start
end
Transcript: hey
Hello!
start
end
start
end


KeyboardInterrupt: 